In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [2]:
def submit_file(predicted_labels,out_file,target='target',index_label='session_id'):
        predicted_df=pd.DataFrame(predicted_labels,index=np.arange(1,predicted_labels.shape[0]+1),columns=[target])
        predicted_df.to_csv(out_file,index_label=index_label)

In [3]:
train_df = pd.read_csv("train_sessions.csv",index_col="session_id")
test_df = pd.read_csv("test_sessions.csv",index_col="session_id")

times = ['time%s' %i for i in range(1,11)]

train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

train_df = train_df.sort_values(by='time1')

train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 253561 entries, 21669 to 204762
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   site1   253561 non-null  int64         
 1   time1   253561 non-null  datetime64[ns]
 2   site2   250098 non-null  float64       
 3   time2   250098 non-null  datetime64[ns]
 4   site3   246919 non-null  float64       
 5   time3   246919 non-null  datetime64[ns]
 6   site4   244321 non-null  float64       
 7   time4   244321 non-null  datetime64[ns]
 8   site5   241829 non-null  float64       
 9   time5   241829 non-null  datetime64[ns]
 10  site6   239495 non-null  float64       
 11  time6   239495 non-null  datetime64[ns]
 12  site7   237297 non-null  float64       
 13  time7   237297 non-null  datetime64[ns]
 14  site8   235224 non-null  float64       
 15  time8   235224 non-null  datetime64[ns]
 16  site9   233084 non-null  float64       
 17  time9   233084 non-null  datet

In [5]:
sites = ['site%s' %i for i in range(1,11)]
train_df[sites].fillna(0).astype('int').to_csv('train_sessions_text.txt',sep=' ',index=None,header=None)
test_df[sites].fillna(0).astype('int').to_csv('test_sessions_text.txt',sep=' ',index=None,header=None)

In [6]:
%%time
cv = CountVectorizer()
with open('train_sessions_text.txt') as inp_train_file:
    x_train = cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    x_test = cv.fit_transform(inp_test_file)
print(x_train.shape,x_test.shape)

(253561, 41592) (82797, 15848)
CPU times: total: 1.77 s
Wall time: 2.81 s


In [7]:
y_train =  train_df['target'].astype('int')

TRAIN LOGISTIC REGRESSION

In [8]:
logit = LogisticRegression(C=0.1,random_state=17)

In [9]:
%%time
cv_score = cross_val_score(logit,x_train,y_train,cv=5,scoring='roc_auc')

CPU times: total: 1.55 s
Wall time: 9 s


In [10]:
cv_score

array([0.90956287, 0.7977546 , 0.85563227, 0.88532845, 0.91547864])

In [11]:
cv_score.mean()

0.8727513666168882